In [17]:
from pathlib import Path

src_root = Path("/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data")
dst_root = Path("/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data_correct")
paths = sorted(src_root.glob("customers/*.json"))
print(len(paths))
paths

764


[PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655293787679.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655293823654.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655295864820.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655296152387.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655296191186.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655296191365.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customers-1655296191557.json'),
 PosixPath('/mnt/sda1/projects/git/courses/udacity_data_engineer/3_lakes/project/data/customers/customer

In [5]:
import json
s = paths[0].read_text()
lst = json.loads(s)
len(lst)

29970

In [8]:
def _find_valid_objects(s):
    return re.findall(r'\{[^{}]*\}', s)

s = paths[1].read_text()
_find_valid_objects(s)

['{"sensorReadingTime":1655471492661,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":264}',
 '{"sensorReadingTime":1655471495694,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":292}',
 '{"sensorReadingTime":1655471498727,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":248}',
 '{"sensorReadingTime":1655471501760,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":240}',
 '{"sensorReadingTime":1655471504793,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":287}',
 '{"sensorReadingTime":1655471507826,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":247}',
 '{"sensorReadingTime":1655471510859,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":242}',
 '{"sensorReadingTime":1655471513892,"serialNumber":"8dc540af-f7fe-4c06-9d5c-a682910d7fdc","distanceFromObject":245}',
 '{"sensorReadingTime":1655471516925,"serialNumb

In [18]:
import json
from tqdm.auto import tqdm
import re

def _form_proper_list(s):
    parts = s.split('}{')
    return '[' + '},{'.join(parts) + ']'

def to_athena_format(src_path: Path):
    """Athena format is one JSON object per line"""
    dst_path = dst_root / src_path.relative_to(src_root)
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    
    s = src_path.read_text()
    objs = _find_valid_objects(s)
    s = "\n".join(objs)
    dst_path.write_text(s)
    

def to_proper_json(src_path: Path):
    print(f"{src_path=}")
    dst_path = dst_root / src_path.relative_to(src_root)
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    
    s = src_path.read_text()
    try:
        json.loads(s)
    except json.JSONDecodeError:
        lst = _find_valid_objects(s)
        s = "[" + ",".join(lst) + "]"
        
        lst = json.loads(s)
        print(f"{len(lst)=} for {src_path=}")
        
        dst_path.write_text(s)
    else:
        raise AssertionError("Wrong assumption?!")

for p in tqdm(paths[:]):
    to_athena_format(p)

  0%|          | 0/764 [00:00<?, ?it/s]